# Prática 06 - Bibliotecas de visualização geográficas

In [1]:
import pandas as pd
import numpy as np
import folium
import folium.plugins as plugins
import geopandas as gpd
import seaborn as sns
import json
import time
import os
import re
import fileinput
from folium.plugins import Search

In [2]:
fp = "data/estados.json"
dadosHomicidios = "data/homicidios_2019.csv"
#fp = "data/municipios_simp.json"
gdf = gpd.read_file(fp)
homicidios = pd.read_csv(dadosHomicidios,delimiter=',')
#gdf = gpd.GeoDataFrame.from_features(customGeoJson)

Aqui fazemos a conversão do objeto `GeoJson` em um [Pandas](www.pandas.pydata.org) `DataFrame`. Posteriormente, determinamos os centroides de cada estado e o centro do mapa

In [3]:
gdf['HOMICIDIOS']=gdf['GEOCODIGO'].astype(int)
gdf['centroid_lon'] = gdf['geometry'].centroid.x
gdf['centroid_lat'] = gdf['geometry'].centroid.y
centerLon = np.mean(gdf['centroid_lon'])
centerLat = np.mean(gdf['centroid_lat'])
numEstados = len(gdf)
numEstados

27

In [12]:
gdf['HOMICIDIOS'] = homicidios['NumHom']

Usamos os dados `centerLon` e `centerLat` para definir as coordenadas centrais do mapa:

In [13]:
mapa = folium.Map(location=[centerLat,centerLon],zoom_start=5,control_scale=True)

In [14]:
g1 = folium.Choropleth(
    geo_data=fp,
    name='choropleth',
    data=gdf,
    columns=['GEOCODIGO', 'HOMICIDIOS'],
    key_on='feature.properties.GEOCODIGO',
    fill_color='RdGy',
    fill_opacity=0.7,
    line_opacity=0,
    bins=6,
    smooth_factor=2,
    legend_name='HOMICIDIOS EM 2019',
    highlight=True
).add_to(mapa)

`GeoJson` para dar **Highlight** nos objetos selecionados

In [15]:
g1 = folium.GeoJson(
        gdf[['NOME', 'geometry','GEOCODIGO']].to_json(), 
        name='Teste Mapa',
        show=True,
        style_function=lambda feature: {
            'fillColor': 'green',
            'color': 'white',
            'weight': 0,
            'fillOpacity':0
        },   
        highlight_function=lambda x: {'weight':0.1, 
                                      'color':'black',
                                      'fillOpacity':0.2},    
        tooltip=folium.features.GeoJsonTooltip(
            fields=['NOME','GEOCODIGO'],
            aliases=['Cidade:','Código'],                              
        ),  
).add_to(mapa)

In [16]:
citysearch = Search(
    layer=g1,
    geom_type='Point',
    placeholder='Busca Cidade',
    collapsed=True,
    search_label='NOME'
).add_to(mapa)

In [10]:
mapa.save("mapaT.html") 

In [17]:
mapa